In [2]:
# Import the modules

from sklearn.externals import joblib
from sklearn import datasets
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np
from collections import Counter
from sklearn.datasets import load_digits
import csv
import random
import math
import operator
import matplotlib.pyplot as plt
from termcolor import colored,cprint

# Import datasets, classifiers and performance metrics

from sklearn.datasets import load_iris
from sklearn import datasets, svm, metrics
from sklearn.cross_validation import KFold
import cv2

# K Nearest Neighbours Implementation

In [3]:
#####################  Euclidean Distance   #########################

def euclideanDistance(x1, x2, length):
    distance = 0
    for x in range(length):
        distance += pow((x1[x] - x2[x]), 2)
    return math.sqrt(distance)

In [4]:
#####################  FInd the Neighbors   #########################


def Find_Neighbors(trainingSet, testData, k):
    distances = []
    
    length = len(testData)-1
    for x in range(len(trainingSet)):
        dist = euclideanDistance(testData, trainingSet[x], length)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

In [5]:
######################### Predict the data   ########################

def Predict(neighbors):
    class_votes = {}
    for x in range(len(neighbors)):
        response = neighbors[x][-1]
        if response in class_votes:
            class_votes[response] += 1
        else:
            class_votes[response] = 1
    sorted_votes = sorted(class_votes.iteritems(), key=operator.itemgetter(1), reverse=True)
    return sorted_votes[0][0]

In [6]:
#########################  Find Accuracy of My model     ############################


def Find_Accuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        
        if testSet[x] == predictions[x]:
            correct += 1
            print colored(">>Predicted Result {0} and Actual Result {1}" \
                          .format(predictions[x],testSet[x]),'green')
    
        else:
            #pass
            cprint(">>Predicted Result {0} and Actual Result {1}" \
                          .format(predictions[x],testSet[x]),None,'on_red')

    return ((correct/float(len(testSet))) * 100.0),correct

# MNIST Handwritten Digit Training Part

In [7]:
##############################  MNIST Digit Training Part for HOG Features  #####################################

# Hog Fetures are used for Object Detection. So as our project is about Handwritten digit recognition, we have 
#     used Hog feature.  


# digits = load_digits()
# np.unique(digits.target)

# features = np.array(digits.data, 'int16') 
# labels = np.array(digits.target, 'int')
# #print features[0].shape

# list_hog_fd = []
# for feature in features:
#     fd = hog(feature.reshape((8, 8)), orientations = 9, pixels_per_cell=(4, 4), \
#              cells_per_block=(1, 1), visualise=False)
#     list_hog_fd.append(fd)
# hog_features = np.array(list_hog_fd, 'float64')



dataset = datasets.fetch_mldata("MNIST Original")

# Extract the features and labels
features = np.array(dataset.data, 'int16') 
labels = np.array(dataset.target, 'int')

# print features[0].shape
# Extract the hog features
list_hog_fd = []
for feature in features:
    # Hog Feature for object Identification
    fd = hog(feature.reshape((28, 28)), orientations=9, pixels_per_cell=(14, 14), \
             cells_per_block=(1, 1), visualise=False)
    list_hog_fd.append(fd)
hog_features = np.array(list_hog_fd, 'float64')

# Count the number of Samples for every digit
print "Count of digits in dataset", Counter(labels)
accuracy = 0.0

X,Y = hog_features,labels
Y = Y.reshape(Y.shape[0],1)

Count of digits in dataset Counter({1: 7877, 7: 7293, 3: 7141, 2: 6990, 9: 6958, 0: 6903, 6: 6876, 8: 6825, 4: 6824, 5: 6313})


In [20]:
X.shape,Y.shape

((70000, 36), (70000, 1))

# Testing Part for Handwriten Digit

In [10]:
X_Train, X_Test = X[:60000], X[69900:]
Y_Train, Y_Test = Y[:60000], Y[69900:]


print Y_Train.shape,X_Test.shape

(60000, 1) (100, 36)


In [16]:
y_predictions = []
y_test = []

trainingSet = np.concatenate((X_Train, Y_Train), axis=1)
testSet = np.concatenate((X_Test, Y_Test), axis=1)
predictions=[]
k = 3

for x in range(len(testSet)):
    neighbors = Find_Neighbors(trainingSet, testSet[x], k)

    result = Predict(neighbors)

    y_predictions.append(repr(result))
    y_test.append(repr(testSet[x][-1]))
    
accuracy,correct = Find_Accuracy(y_test, y_predictions)

print "\n\n" + str(correct) + " test cases classified correctly out of " + \
        str(len(y_test)) + " with an efficiency of " + \
            str(accuracy) + " %"

>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 7.0 and Actual Result 9.0
>>Predicted Result 8.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicte

In [25]:
X.shape,Y.shape

((70000, 36), (70000, 1))

# 1st Handwritten Digit Testing Image

In [21]:
########################################  Photo 1 Testing part    ########################################## 

# Read the input image 

im = cv2.imread("photo_1.jpg")

# Convert to grayscale and apply Gaussian filtering
im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
im_gray = cv2.GaussianBlur(im_gray, (3, 3), 0)

############## Show the Original Image  ###################

cv2.imshow("Original Image",im)

# Threshold the image
ret, im_th = cv2.threshold(im_gray, 90, 255, cv2.THRESH_BINARY_INV)

# Find contours in the image
ctrs, hier = cv2.findContours(im_th.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

testingSet = [cv2.boundingRect(ctr) for ctr in ctrs]

# Training the MNIST dataset
trainingSet = np.concatenate((X, Y), axis=1)

print "Wait for a moment......"

print "\n\nMy Predicted Result \n"

for rect in testingSet:
    
    # Draw the rectangle around the digit..
    cv2.rectangle(im, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (0, 255, 0), 3) 
    
    leng = int(rect[3] * 1.6)
    pt1 = int(rect[1] + rect[3] // 2 - leng // 2)
    pt2 = int(rect[0] + rect[2] // 2 - leng // 2)
    roi = im_th[pt1:pt1+leng, pt2:pt2+leng]
    
    roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)
    
    roi = cv2.dilate(roi, (4, 4))
    
    roi_hog_fd = hog(roi, orientations=9, pixels_per_cell=(14, 14), cells_per_block=(1, 1), visualise=False)
    
    k = 3
    
    #########################  Implementation of Algorithm   ##################################
    
    neighbors = Find_Neighbors(trainingSet, roi_hog_fd, k)
    
    result = Predict(neighbors)
    
    print result
    
    cv2.putText(im, str(int(result)), (rect[0], rect[1]),cv2.FONT_HERSHEY_DUPLEX, 2, (0, 255, 255), 3)
    
cv2.imshow("Resulting Image", im)
cv2.waitKey(0)
cv2.destroyAllWindows()

Wait for a moment......


My Predicted Result 

2.0
5.0
0.0
1.0
2.0
8.0
9.0
0.0


# One More Testing Image...

In [ ]:
####################################  Harsh Handwritting Testing part    #################################### 

# Read the input image 

im = cv2.imread("harsh.png")

# Convert to grayscale and apply Gaussian filtering
im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
im_gray = cv2.GaussianBlur(im_gray, (3, 3), 0)

############## Original Image  ###################

cv2.imshow("Original Image",im)

# Threshold the image
ret, im_th = cv2.threshold(im_gray, 150, 255, cv2.THRESH_BINARY_INV)

# Find contours in the image
cv2.imshow("Threshold",im_th)
ctrs, hier = cv2.findContours(im_th.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

testingSet = [cv2.boundingRect(ctr) for ctr in ctrs]

trainingSet = np.concatenate((X, Y), axis=1)

print "Wait for a moment......"

print "\n\nMy Predicted Result \n"

for rect in testingSet:
    
    cv2.rectangle(im, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (0, 255, 0), 3) 
    
    leng = int(rect[3] * 1.6)
    pt1 = int(rect[1] + rect[3] // 2 - leng // 2)
    pt2 = int(rect[0] + rect[2] // 2 - leng // 2)
    roi = im_th[pt1:pt1+leng, pt2:pt2+leng]
    
    roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)
    
    roi = cv2.dilate(roi, (4, 4))
    
    roi_hog_fd = hog(roi, orientations=9, pixels_per_cell=(14, 14), cells_per_block=(1, 1), visualise=False)
    #print roi_hog_fd
    k = 3
    
    #########################  Implementation of Algorithm   ##################################
    
    neighbors = Find_Neighbors(trainingSet, roi_hog_fd, k)
    
    result = Predict(neighbors)
    
    print result
    
    cv2.putText(im, str(int(result)), (rect[0], rect[1]),cv2.FONT_HERSHEY_DUPLEX, 1, (255, 0, 0), 2)
    
cv2.imshow("Resulting Image", im)
cv2.waitKey(0)
cv2.destroyAllWindows()

Wait for a moment......


My Predicted Result 

5.0
2.0

# Another Testing Image

In [29]:
###################################   Other Handwritting Digit Testing part    ################################### 

# Read the input image 

im = cv2.imread("parikh.png")

# Convert to grayscale and apply Gaussian filtering
im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
im_gray = cv2.GaussianBlur(im_gray, (3, 3), 0)

############## Original Image  ###################

cv2.imshow("Original Image",im)

# Threshold the image
ret, im_th = cv2.threshold(im_gray, 190, 255, cv2.THRESH_BINARY_INV)

cv2.imshow("Threshold",im_th)
# Find contours in the image
ctrs, hier = cv2.findContours(im_th.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

testingSet = [cv2.boundingRect(ctr) for ctr in ctrs]

trainingSet = np.concatenate((X, Y), axis=1)

print "Wait for a moment......"

print "\n\nMy Predicted Result \n"

for rect in testingSet:
    
    cv2.rectangle(im, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (0, 255, 0), 3) 
    
    leng = int(rect[3] * 1.6)
    pt1 = int(rect[1] + rect[3] // 2 - leng // 2)
    pt2 = int(rect[0] + rect[2] // 2 - leng // 2)
    roi = im_th[pt1:pt1+leng, pt2:pt2+leng]
    
    roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)
    
    roi = cv2.dilate(roi, (5, 5))
    
    roi_hog_fd = hog(roi, orientations=9, pixels_per_cell=(14, 14), cells_per_block=(1, 1), visualise=False)
    
    k = 3

    #########################  Implementation of Algorithm   ##################################
    
    neighbors = Find_Neighbors(trainingSet, roi_hog_fd, k)
    
    result = Predict(neighbors)
    
    print result
    
    cv2.putText(im, str(int(result)), (rect[0], rect[1]),cv2.FONT_HERSHEY_DUPLEX, 1, (255, 0, 0), 2)
    
cv2.imshow("Resulting Image", im)
cv2.waitKey(0)
cv2.destroyAllWindows()

Wait for a moment......


My Predicted Result 

5.0
3.0
7.0
4.0


# Now comes to Letter Recognition Part

In [10]:
##############################     Letter Recognition Training Part    ##################################


import numpy as np
import time
from collections import Counter


# Letter Dataset 
# https://archive.ics.uci.edu/ml/datasets/Letter+Recognition

fname = 'letter-recognition.data'

data = np.loadtxt(fname, np.float32, delimiter=',', converters={ 0 : lambda ch : ord(ch)-ord('A') })

X_letter,Y_letter = data[:,1:],data[:,0]
Y_letter = Y_letter.reshape(Y_letter.shape[0],1)

X_Train, X_Test = X_letter[:19000], X_letter[19900:]
Y_Train, Y_Test = Y_letter[:19000], Y_letter[19900:]


print Y_Train.shape,X_Test.shape

(19000, 1) (100, 16)


In [8]:
##############################     Letter Recognition Testing Part    ##################################



accuracy = 0.0
y_predictions = []
y_test = []

trainingSet = np.concatenate((X_Train, Y_Train), axis=1)
testSet = np.concatenate((X_Test, Y_Test), axis=1)
predictions=[]
k = 3

for x in range(len(testSet)):
    neighbors = Find_Neighbors(trainingSet, testSet[x], k)
    result = Predict(neighbors)
    y_predictions.append(chr(int(result)+65))
    y_test.append(chr(int(testSet[x][-1])+65))
        
accuracy,correct = Find_Accuracy(y_test, y_predictions)

#print "\n\nAccuracy is {0} %".format(accuracy)

print "\n\n" + str(correct) + " test cases classified correctly out of " + \
        str(len(y_test)) + " with an efficiency of " + \
            str(accuracy) + " %"


>>Predicted Result J and Actual Result J
>>Predicted Result V and Actual Result V
>>Predicted Result V and Actual Result V
>>Predicted Result L and Actual Result L
>>Predicted Result H and Actual Result H
>>Predicted Result C and Actual Result C
>>Predicted Result X and Actual Result X
>>Predicted Result Y and Actual Result Y
>>Predicted Result D and Actual Result D
>>Predicted Result U and Actual Result U
>>Predicted Result I and Actual Result I
>>Predicted Result Q and Actual Result Q
>>Predicted Result J and Actual Result J
>>Predicted Result G and Actual Result G
>>Predicted Result U and Actual Result U
>>Predicted Result X and Actual Result X
>>Predicted Result B and Actual Result P
>>Predicted Result T and Actual Result T
>>Predicted Result T and Actual Result T
>>Predicted Result Y and Actual Result Y
>>Predicted Result G and Actual Result G
>>Predicted Result K and Actual Result K
>>Predicted Result S and Actual Result S
>>Predicted Result N and Actual Result N
>>Predicted Resu

# Training for HOG Feature for Letter Dataset

In [17]:
########################  Letter Recognition Training Part for HOG Features #########################


import numpy as np
import time
from collections import Counter


fname = 'letter-recognition.data'

data = np.loadtxt(fname, np.float32, delimiter=',', converters={ 0 : lambda ch : ord(ch)-ord('A') })

X_letter,Y_letter = data[:,1:],data[:,0]

X_letter = np.array(X_letter,'int16')
Y_letter = np.array(Y_letter,'int')

print "Before X Features Dimention"

print X_letter.shape
list_hog_fd = []

for feature in X_letter:
    # Hog Feature for object Identification
    fd = hog(feature.reshape((4, 4)), orientations=9, pixels_per_cell=(2, 2), \
             cells_per_block=(1, 1), visualise=False)
    list_hog_fd.append(fd)
hog_features = np.array(list_hog_fd, 'float64')

accuracy = 0.0

print "\nAfter X Features Dimention"

print hog_features.shape

X_letter = hog_features
Y_letter = Y_letter.reshape(Y_letter.shape[0],1)

Before X Features Dimention
(20000, 16)

After X Features Dimention
(20000, 36)


In [31]:
X_letter.shape,Y_letter.shape

((20000, 36), (20000, 1))

# First Testing Data for Letter Recognition 

In [37]:
##################################   Handwritten Letters Testing part    #################################### 

# Read the input image 

im = cv2.imread("dipen_letter.png")

# Convert to grayscale and apply Gaussian filtering
im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
im_gray = cv2.GaussianBlur(im_gray, (3, 3), 0)

############## Original Image  ###################

cv2.imshow("Original Image",im)

ret, im_th = cv2.threshold(im_gray, 220, 255, cv2.THRESH_BINARY_INV)
cv2.imshow("Threshold",im_th)

# Find contours in the image
ctrs, hier = cv2.findContours(im_th.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

testingSet = [cv2.boundingRect(ctr) for ctr in ctrs]

trainingSet = np.concatenate((X_letter, Y_letter), axis=1)

print "Wait for a moment......"

print "\n\nMy Predicted Result \n"

for rect in testingSet:
    
    cv2.rectangle(im, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (0, 255, 0), 3) 
    
    leng = int(rect[3] * 1.6)
    pt1 = int(rect[1] + rect[3] // 2 - leng // 2)
    pt2 = int(rect[0] + rect[2] // 2 - leng // 2)
    roi = im_th[pt1:pt1+leng, pt2:pt2+leng]
    
    roi = cv2.resize(roi, (4, 4), interpolation=cv2.INTER_AREA)
    
    roi = cv2.dilate(roi, (3, 3))
    
    roi_hog_fd = hog(roi, orientations=9, pixels_per_cell=(2, 2), cells_per_block=(1, 1), visualise=False)

    k = 3
    
    #########################  Implementation of Algorithm   ##################################
    
    neighbors = Find_Neighbors(trainingSet, roi_hog_fd, k)
    
    result = Predict(neighbors)
    print str(chr(int(result)+65))
    cv2.putText(im, chr(int(result)+65), (rect[0], rect[1]),cv2.FONT_HERSHEY_DUPLEX, 1, (255, 0, 0), 2)
    
cv2.imshow("Resulting Image", im)
cv2.waitKey(0)
cv2.destroyAllWindows()

Wait for a moment......


My Predicted Result 

B
K
N
A
A
Q
B
A
D
E
W
O
G


# KNN Inbuilt Method

In [7]:
#########################  Inbuilt Method for KNN    ##########################

# http://brianfarris.me/static/digit_recognizer.html


x_train, x_test = X[:60000], X[60000:]
y_train, y_test = Y[:60000], Y[60000:]


clf_knn = KNeighborsClassifier()
clf_knn.fit(x_train, y_train)
y_pred_knn = clf_knn.predict(x_test)
acc_knn = accuracy_score(y_test, y_pred_knn)
print "K Nearest Neighbors Accuracy: ",(acc_knn*100), "%"

K Nearest Neighbors Accuracy:  90.41 %


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
